In [1]:
print("Hi! I'm a Python script.")

Hi! I'm a Python script.


In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Connect to Trino (adjust connection details as needed)
engine = create_engine("trino://user@localhost:9090/memory", connect_args={"user": "user"})

In [4]:
# Create a df named taxi_df with fake data
df = pd.DataFrame({
    "vendor_id": [1, 2, 2, 1, 2],
    "pickup_datetime": ["2021-01-01 00:00:00", "2021-01-01 00:10:00", "2021-01-01 00:20:00", "2021-01-01 00:30:00", "2021-01-01 00:40:00"],
    "dropoff_datetime": ["2021-01-01 00:00:00", "2021-01-01 00:10:00", "2021-01-01 00:20:00", "2021-01-01 00:30:00", "2021-01-01 00:40:00"],
    "passenger_count": [1, 2, 3, 4, 5],
    "trip_distance": [1.0, 2.0, 3.0, 4.0, 5.0],
    "rate_code": [1, 2, 3, 4, 5],
    "store_and_fwd_flag": [True, False, True, False, True],
    "payment_type": ["credit", "cash", "credit", "cash", "credit"],
    "fare_amount": [10.0, 20.0, 30.0, 40.0, 50.0]
})

In [5]:
df

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount
0,1,2021-01-01 00:00:00,2021-01-01 00:00:00,1,1.0,1,True,credit,10.0
1,2,2021-01-01 00:10:00,2021-01-01 00:10:00,2,2.0,2,False,cash,20.0
2,2,2021-01-01 00:20:00,2021-01-01 00:20:00,3,3.0,3,True,credit,30.0
3,1,2021-01-01 00:30:00,2021-01-01 00:30:00,4,4.0,4,False,cash,40.0
4,2,2021-01-01 00:40:00,2021-01-01 00:40:00,5,5.0,5,True,credit,50.0


In [6]:
# Store the data in Trino (using the schema named “default”):
df.to_sql(con=engine, name="taxi", schema="default", method="multi", index=False)
engine.dispose()

In [22]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [23]:
%reload_ext sql
%config SqlMagic.named_parameters="enabled"

In [24]:
%sql trino://user@localhost:9090/memory

In [25]:
%sqlcmd tables --schema default

Name
taxi


In [26]:
%sqlcmd columns --table taxi --schema default

name,type,nullable,default
vendor_id,BIGINT,True,None
pickup_datetime,VARCHAR,True,None
dropoff_datetime,VARCHAR,True,None
passenger_count,BIGINT,True,None
trip_distance,DOUBLE,True,None
rate_code,BIGINT,True,None
store_and_fwd_flag,BOOLEAN,True,None
payment_type,VARCHAR,True,None
fare_amount,DOUBLE,True,None


In [27]:
# Run a query to check the data
%sql SELECT * FROM default.taxi

Running query in 'trino://user@localhost:9090/memory'

vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount
1,2021-01-01 00:00:00,2021-01-01 00:00:00,1,1.0,1,True,credit,10.0
2,2021-01-01 00:10:00,2021-01-01 00:10:00,2,2.0,2,False,cash,20.0
2,2021-01-01 00:20:00,2021-01-01 00:20:00,3,3.0,3,True,credit,30.0
1,2021-01-01 00:30:00,2021-01-01 00:30:00,4,4.0,4,False,cash,40.0
2,2021-01-01 00:40:00,2021-01-01 00:40:00,5,5.0,5,True,credit,50.0


In [15]:
%sql SELECT * FROM default.taxi WHERE payment_type = 'credit'

Running query in 'trino://user@localhost:9090/memory'

vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount
1,2021-01-01 00:00:00,2021-01-01 00:00:00,1,1.0,1,True,credit,10.0
2,2021-01-01 00:20:00,2021-01-01 00:20:00,3,3.0,3,True,credit,30.0
2,2021-01-01 00:40:00,2021-01-01 00:40:00,5,5.0,5,True,credit,50.0


In [30]:
type_of_payment='cash'
%sql SELECT * FROM default.taxi WHERE payment_type = :type_of_payment

Running query in 'trino://user@localhost:9090/memory'

vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount
2,2021-01-01 00:10:00,2021-01-01 00:10:00,2,2.0,2,False,cash,20.0
1,2021-01-01 00:30:00,2021-01-01 00:30:00,4,4.0,4,False,cash,40.0


In [32]:
%%sql
SELECT *
FROM default.taxi
LIMIT 2

Running query in 'trino://user@localhost:9090/memory'

vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount
1,2021-01-01 00:00:00,2021-01-01 00:00:00,1,1.0,1,True,credit,10.0
2,2021-01-01 00:10:00,2021-01-01 00:10:00,2,2.0,2,False,cash,20.0
